In [1]:
import platform
import os
import cloudpickle
import torch
import pandas as pd
from diffusers import EulerDiscreteScheduler
from diffusers import StableDiffusionPipeline
from verta import Client
from verta.dataset import Path
from verta.dataset.entities import Dataset
from verta.dataset.entities import DatasetVersion
from verta.deployment import DeployedModel
from verta.registry import VertaModelBase, verify_io, task_type
from verta.registry.entities import RegisteredModel, RegisteredModelVersion
from verta.environment import Python
from verta.registry import data_type
from verta.endpoint import Endpoint

from verta.utils import ModelAPI

In [19]:
os.environ['VERTA_EMAIL'] = 'cory@verta.ai'
os.environ['VERTA_DEV_KEY'] = '154a34ad-2cd2-4d5d-b87c-2b809e075faa'
os.environ['VERTA_HOST'] = 'cj.dev.verta.ai'
client: Client = Client()
project_name = "Stable Diffusion v2 Example"
endpoint_name = "Stable_Diffusion_v2_Example"
dataset_name = "Stable Diffusion v2 prebuilt pipeline"
version = "v11"

got VERTA_HOST from environment
got VERTA_EMAIL from environment
got VERTA_DEV_KEY from environment
connection successfully established


In [3]:
# create the local data directory to store the prebuilt assets
os.makedirs(
    os.path.dirname('data/'),
    exist_ok=True,
)

In [4]:
print("configuring scheduler")
model_id = "stabilityai/stable-diffusion-2-1"
scheduler: EulerDiscreteScheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

print("configuring pipeline")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
revision = 'fp16'
torch_dtype = torch.float16

processor = platform.processor()
# initialize the image pipeline using custom setting for M1/M2 mac
if processor == 'arm':
    # ARM-based Macs do not support the fp16 revision nor the float16 dtype when initializing the pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        scheduler=scheduler
    )
else:
    # x86 based OSes can use the fp16 revision and float16 dtype
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        scheduler=scheduler,
        torch_dtype=torch_dtype,
        revision=revision,
    )

print("configuring pipeline device")

pipe: StableDiffusionPipeline = pipe.to(device)


configuring scheduler
configuring pipeline


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

configuring pipeline device


In [0]:
# Store the pretrained model to disk
print("storing pretrained pipeline to disk")
StableDiffusionPipeline.save_pretrained(pipe, save_directory='./data/pipe')

In [6]:
# create a dataset out of the pretrained model
print("creating a dataset for the pretrained pipeline")
dataset: Dataset = client.get_or_create_dataset(name=dataset_name)

creating a dataset for the pretrained pipeline
got existing Dataset: Stable Diffusion v2 prebuilt pipeline


In [7]:
content: Path = Path([f"./data/pipe"], enable_mdb_versioning=True)
# dataset_version: DatasetVersion = dataset.create_version(content)
dataset_version: DatasetVersion = dataset.get_latest_version()

got existing dataset version: 07bb18385e80f1b5510f3ed1bad8f8fc1e2fb0ee2bf4f269dc914c00ebb2895c


In [8]:
# define a custom verta model that will use prebuilt pipeline in the dataset to run the prediction
class StableDiffusionV2Generator(VertaModelBase):
    def __init__(self, artifacts):
        self.model = cloudpickle.load(open(artifacts["serialized_model"], "rb"))
        components = dataset_version.list_components()
        for component in components:
            print("data set component {}".format(component))
        self.pipeline = StableDiffusionPipeline.from_pretrained('./data/pipe')

    @verify_io
    def predict(self, prompt):
        num_images = 1
        image_length = 768
        guidance_scale = 9
        num_inference_steps = 25
        images = self.pipeline(
            prompt,
            num_images_per_prompt=num_images,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            height=image_length,
            width=image_length,
        ).images
        return [images[0], image_length, image_length, guidance_scale, num_images]

In [9]:
# create the registered model
print("configuring verta model")
registered_model: RegisteredModel = client.get_or_create_registered_model(name=project_name, data_type=data_type.Image(), task_type=task_type.Other())

configuring verta model
got existing RegisteredModel: Stable Diffusion v2 Example


In [10]:
# build the model api
model_api: ModelAPI = ModelAPI(
    pd.DataFrame.from_records(
        [{"prompt": "the prompt"}]),
    pd.DataFrame.from_records([{"image_data": "", "image_height": 768, "image_width": 768, "guidance_scale": 9, "num_inference_steps": 25}]),
)

In [20]:

# create the model version
model_version: RegisteredModelVersion = registered_model.create_standard_model(
    name=version,
    model_cls=StableDiffusionV2Generator,
    model_api = model_api,
    environment=Python(requirements=Python.read_pip_file("requirements.txt"))
)

# log the dataset version that contains the prebuilt pipeline
model_version.log_dataset_version(key=dataset_name, dataset_version=dataset_version)

created new ModelVersion: v11
uploading model to Registry
uploading part 1
upload complete
uploading model_api.json to Registry
uploading part 1
upload complete
uploading custom_modules to Registry
uploading part 1
upload complete


In [21]:
# model_version: RegisteredModelVersion = registered_model.get_or_create_version(name=version)

got existing RegisteredModelVersion: v11


/Users/cory/.pyenv/versions/verta3.9.14/lib/python3.9/site-packages/verta/_internal_utils/_utils.py:1457: UserWarning: Model Version with name v11 already exists; cannot set `desc`, `labels`, `attrs`, `time_created`, `lock_level`, `input_description`, `hide_input_label`, `output_description`, or `hide_output_label`
  warnings.warn(


In [ ]:
endpoint: Endpoint = client.get_or_create_endpoint(endpoint_name)
endpoint.update(model_version, wait=True)

waiting for update...................................

In [ ]:
# generate an image from a text prompt
deployed_model: DeployedModel = endpoint.get_deployed_model()
prediction = deployed_model.predict(["An artistic logo for an AI company named Verta"])
